# Project #1 - Data Wrangling and Regression Analysis

Name: Ronaldo Aparecido de Oliveira                
ID: 300305192

## Part1 - Data Collection and Preparation

### Step 1 - Read the process the Fortune1000 data file

In [1]:
from bs4 import BeautifulSoup as bs
from decimal import Decimal

import pandas as pd
import numpy as np
import requests
import sys
import time
import numbers

In [2]:
fortune1000 = []

with open("fortune1000.txt", "r") as infile:
    # loop over each line of the file - ignore blank lines
    for line in infile:
        if (line != '\n'):
            fortune1000.append(line)

In [3]:
len(fortune1000)

11000

In [4]:
# columns of the dataset
allColumns = ['Rank', 'Name', 'Revenue($m)', '% Change in Revenue', 'Profits($m)', '% Change in Profits', 
           'Assets($m)', 'Market Value($m)', 'Change in Rank(1000)', 'Employees', 'Change in Rank (500 only)']
badColumns = ['% Change in Revenue', '% Change in Profits', 'Change in Rank(1000)', 'Change in Rank (500 only)']
goodColumns = ['Rank', 'Name', 'Revenue($m)', 'Profits($m)', 'Assets($m)', 'Market Value($m)', 'Employees']

In [5]:
# create the data frame - reshaping the list to a dataframe[1000, 11]
fortune1000 = pd.DataFrame(np.reshape(fortune1000, (1000, 11)), columns=allColumns)
fortune1000.head()

,Rank,Name,Revenue($m),% Change in Revenue,Profits($m),% Change in Profits,Assets($m),Market Value($m),Change in Rank(1000),Employees,Change in Rank (500 only)
0,1\n,Walmart\n,"$523,964\n",1.9%\n,"$14,881\n",123.1%\n,"$236,495\n","$321,803.3\n",-\n,"2,200,000\n",-\n
1,2\n,Amazon.com\n,"$280,522\n",20.5%\n,"$11,588\n",15%\n,"$225,248\n","$970,680.1\n",3\n,"798,000\n",3\n
2,3\n,Exxon Mobil\n,"$264,938\n",-8.7%\n,"$14,340\n",-31.2%\n,"$362,597\n","$160,696.3\n",-1\n,"74,900\n",-1\n
3,4\n,Apple\n,"$260,174\n",-2%\n,"$55,256\n",-7.2%\n,"$338,516\n","$1,112,640.8\n",-1\n,"137,000\n",-1\n
4,5\n,CVS Health\n,"$256,776\n",32%\n,"$6,634\n",-\n,"$222,449\n","$77,375.8\n",3\n,"290,000\n",3\n


In [6]:
# delete columns that is not necessary
fortune1000 = fortune1000.drop(columns=badColumns, axis="1")
fortune1000.head()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees
0,1\n,Walmart\n,"$523,964\n","$14,881\n","$236,495\n","$321,803.3\n","2,200,000\n"
1,2\n,Amazon.com\n,"$280,522\n","$11,588\n","$225,248\n","$970,680.1\n","798,000\n"
2,3\n,Exxon Mobil\n,"$264,938\n","$14,340\n","$362,597\n","$160,696.3\n","74,900\n"
3,4\n,Apple\n,"$260,174\n","$55,256\n","$338,516\n","$1,112,640.8\n","137,000\n"
4,5\n,CVS Health\n,"$256,776\n","$6,634\n","$222,449\n","$77,375.8\n","290,000\n"


In [7]:
# clean up data - delete '$', '\n', ','
fortune1000 = fortune1000[goodColumns].replace({'\$': '', '\\n': '', ',': ''}, regex = True)
fortune1000.head()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees
0,1,Walmart,523964,14881,236495,321803.3,2200000
1,2,Amazon.com,280522,11588,225248,970680.1,798000
2,3,Exxon Mobil,264938,14340,362597,160696.3,74900
3,4,Apple,260174,55256,338516,1112640.8,137000
4,5,CVS Health,256776,6634,222449,77375.8,290000


In [8]:
fortune1000.tail()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees
995,996,Mr. Cooper Group,2007,274,18305,674.1,9100
996,997,Herc Holdings,1999,47.5,3817,590.5,5100
997,998,Healthpeak Properties,1997.4,45.5,14032.9,12059.3,204
998,999,SPX FLOW,1996.3,-95.1,2437.4,1211.8,5000
999,1000,Liberty Oilfield Services,1990.3,39,1283.4,302.8,2571


In [9]:
fortune1000.describe()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees
count,1000,1000,1000,1000,1000,1000,1000
unique,997,1000,993,960,996,946,730
top,457,Belden,6146,96,6950,-,12000
freq,2,1,2,3,2,50,8


In [10]:
fortune1000.to_csv('dfFortune1000.csv')

### Step 2 - Add a column of Company URLs to the DataFrame

In [11]:
# URL with all company websites
url = 'http://gist.github.com/hrbrmstr/ae574201af3de035c684'

# request a connection to server
response = requests.get(url)
# content returned
soup = bs(response.content, "html.parser")

In [12]:
# store all websites from web page
company_name = []
websites = []

for i in range(2,1002):
    
    # create the 'id' to be used to find the value and use the 'td' tag to get all values
    cells = soup.find(id='file-f1000-csv-LC' + str(i)).find_all('td')
    
    # get the website information
    company_name.append(cells[1].get_text())
    websites.append(cells[4].get_text())

In [13]:
len(company_name)

1000

In [14]:
# create a new data frame with the extract information from gitHub
Company_Website = pd.DataFrame(zip(company_name, websites), columns=['Name', 'Website'])
Company_Website.head()

,Name,Website
0,Walmart,http://www.walmart.com
1,Exxon Mobil,http://www.exxonmobil.com
2,Chevron,http://www.chevron.com
3,Berkshire Hathaway,http://www.berkshirehathaway.com
4,Apple,http://www.apple.com


In [15]:
# insert the website column to original data frame - created a new Data Frame
newFortune1000 = fortune1000.merge(Company_Website, on='Name', how='left')
newFortune1000.head()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees,Website
0,1,Walmart,523964,14881,236495,321803.3,2200000,http://www.walmart.com
1,2,Amazon.com,280522,11588,225248,970680.1,798000,http://www.amazon.com
2,3,Exxon Mobil,264938,14340,362597,160696.3,74900,http://www.exxonmobil.com
3,4,Apple,260174,55256,338516,1112640.8,137000,http://www.apple.com
4,5,CVS Health,256776,6634,222449,77375.8,290000,http://www.cvshealth.com


In [16]:
newFortune1000[newFortune1000['Website'].isnull()].head()

,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees,Website
13,14,Costco Wholesale,152703,3659,45400,125907.7,201500,NaN
16,17,JPMorgan Chase,142422,36431,2687379,276750.1,256981,NaN
18,19,Walgreens Boots Alliance,136866,3982,67598,40528.2,287000,NaN
19,20,Verizon Communications,131868,19265,291727,222220,135000,NaN
24,25,Bank of America,113589,27430,2434079,185226.9,208131,NaN


In [17]:
# total of companies where website was not found
newFortune1000['Website'].isnull().sum()

290

In [18]:
# delete the row with null in 'website' columns
newFortune1000.dropna(inplace=True)

# save the new data frame - cleaned
newFortune1000.to_csv('dfNewFortune1000.csv')

In [19]:
newFortune1000.shape

(710, 8)

In [20]:
newFortune1000['Website'][:5]

0       http://www.walmart.com
1        http://www.amazon.com
2    http://www.exxonmobil.com
3         http://www.apple.com
4     http://www.cvshealth.com
Name: Website, dtype: object

### Step 3 - Extract/Scrape search Advertising Related Data from a website

In [21]:
# Suffic URL to be used to find the statistics
url_suffix = ".websiteoutlook.com"

# variables to store the data
company_website = []
alexa_rank = []
back_links = []
page_authority = []
domain_authority = []
moz_rank = []
pageviews = []
worth = []
page_size = []
code_text_ratio = []
semrush_rank = []
keywords = []
organic_traffic = []
cost_in_usd = []
adwords_keywords = []
adwords_traffic = []
adwords_buget_in_usd = []

counter = 0

In [22]:
newFortune1000a = ['http://www.tegna.com']

# loop over all companies
for url_prefix in newFortune1000['Website']:

    company_website.append(url_prefix)
    url = []
    
    #print('Company Bef 1=> ' + url_prefix)

    if (url_prefix[-1] == '/'):
        url_prefix = url_prefix[:-1]

    #print('Company Bef 2=> ' + url_prefix)
    
    try:
        # create the URL
        if (url_prefix[7:10].strip() == 'www'):
            url = url_prefix[0:7] + url_prefix[11:] + url_suffix
            #print('Case 1 => ' + url)
        elif (url_prefix[:3].strip() == 'www'):
            url = "http://" + url_prefix[4:] + url_suffix
            #print('Case 2 => ' + url)
        else:
            url = url_prefix + url_suffix
        
        #print('Company Aft => ' + url)
        
        # request a connection to server
        response = requests.get(url)
        # content returned
        soup = bs(response.content, "html.parser")

        #print('alexa_rank: ' + soup.find('span', {'class': 'label label-primary'}).get_text())
        #print('back_links: ' + soup.find('span', {'class': 'label label-default'}).get_text())

        # extract Basic Information
        alexa_rank.append(soup.find('span', {'class': 'label label-primary'}).get_text())
        back_links.append(soup.find('span', {'class': 'label label-default'}).get_text())
        
        # these 3 values have the same 'class' - using index
        item = soup.find_all('span', {'class': 'label label-info'})
        
        #print('page_authority: ' + item[0].get_text())
        #print('domain_authority: ' + item[1].get_text())
        #print('moz_rank: ' + item[2].get_text())
        
        page_authority.append(item[0].get_text())
        domain_authority.append(item[1].get_text())
        moz_rank.append(item[2].get_text())
            
        #print('pageviews: ' + soup.find('span', {'class': 'label label-warning'}).get_text())
        #print('worth: ' + soup.find('span', {'class': 'label label-danger'}).get_text())
        
        pageviews.append(soup.find('span', {'class': 'label label-warning'}).get_text())
        worth.append(soup.find('span', {'class': 'label label-danger'}).get_text())
        
        # select all 'class: panel-heading' - there are many in the document
        panel_headings = soup.find_all('div', {'class': 'panel-heading'})

        # loop over all 'class = panel-heading'
        for panel_heading in panel_headings:
            
            # extract Website Information
            if (panel_heading.get_text() == 'Website Information'):
                
                text = panel_heading.findNext('div', {'class': 'panel-body'}).get_text()
                
                if text.strip() != 'site not available':
                    #print('page_size: ' + soup.find('dt', text='Page Size').findNext('dd').get_text())
                    #print('code_text_ratio: ' + soup.find('dt', text='Code to Text Ratio').findNext('dd').get_text())
                    
                    # extract Web Information
                    page_size.append(soup.find('dt', text='Page Size').findNext('dd').get_text())
                    code_text_ratio.append(soup.find('dt', text='Code to Text Ratio').findNext('dd').get_text())
                else:
                    #print('not information ')

                    page_size.append(None)
                    code_text_ratio.append(None)

            
            # extract SemRush Methics
            if (panel_heading.get_text() == 'SemRush Metrics') :

                text = panel_heading.findNext('div', {'class': 'panel-body'}).get_text()

                # check if there is information avaliable in the section - if no, fill with 'none'
                if text.strip() != 'No Data availableView Full Report':
                    
                    #print('semrush_rank: ' + soup.find('td', text='Semrush Rank').findNext('td').get_text())
                    #print('keywords: ' + soup.find('td', text='Keywords').findNext('td').get_text())
                    #print('organic_traffic: ' + soup.find('td', text='Organic Traffic').findNext('td').get_text())
                    #print('cost_in_usd: ' + soup.find('td', text='Cost (in USD)').findNext('td').get_text())           
                    #print('adwords_keywords: ' + soup.find('td', text='Adwords Keyword').findNext('td').get_text())
                    #print('adwords_traffic: ' + soup.find('td', text='Adwords Traffic').findNext('td').get_text())
                    #print('adwords_buget_in_usd: ' + soup.find('td', text='Adwords budget (in USD)').findNext('td').
                    #get_text())
                    
                    semrush_rank.append(soup.find('td', text='Semrush Rank').findNext('td').get_text())
                    keywords.append(soup.find('td', text='Keywords').findNext('td').get_text())
                    organic_traffic.append(soup.find('td', text='Organic Traffic').findNext('td').get_text())
                    cost_in_usd.append(soup.find('td', text='Cost (in USD)').findNext('td').get_text())
                    adwords_keywords.append(soup.find('td', text='Adwords Keyword').findNext('td').get_text())
                    adwords_traffic.append(soup.find('td', text='Adwords Traffic').findNext('td').get_text())
                    adwords_buget_in_usd.append(soup.find('td', text='Adwords budget (in USD)').findNext('td').get_text())
                else:
                    
                    #print('semrush_rank: None')
                    
                    semrush_rank.append(None)
                    keywords.append(None)
                    organic_traffic.append(None)
                    cost_in_usd.append(None)
                    adwords_keywords.append(None)
                    adwords_traffic.append(None)
                    adwords_buget_in_usd.append(None)
                    
            #print('OK - Company: ' + url_prefix)
        
    except:
        # if an exception is thrown, each information is filled with None
        alexa_rank.append(None)
        back_links.append(None)
        page_authority.append(None)
        domain_authority.append(None)
        moz_rank.append(None)
        pageviews.append(None)
        worth.append(None)
        page_size.append(None)
        code_text_ratio.append(None)
        semrush_rank.append(None)
        keywords.append(None)
        organic_traffic.append(None)
        cost_in_usd.append(None)
        adwords_keywords.append(None)
        adwords_traffic.append(None)
        adwords_buget_in_usd.append(None)
        
        #print("URL not found - ", url)
        continue
           
    counter +=1
    
    # wait 1 second to next request
    time.sleep(0.2)
    

In [23]:
# create the data frame with all information
extractedData = pd.DataFrame(zip(company_website, alexa_rank, back_links, page_authority, domain_authority, moz_rank, 
                                 pageviews, worth, page_size, code_text_ratio, semrush_rank, keywords, organic_traffic, 
                                 cost_in_usd, adwords_keywords, adwords_traffic, adwords_buget_in_usd),
                            columns=['Website', 'Alexa Rank', 'Back Links', 'Page Authority', 'Domain Authority', 
                                     'Moz Rank', 'Pageviews', 'Worth', 'Page Size', 'Code to Text Ratio', 'Semrush Rank', 
                                     'Keywords', 'Organic Traffic', 'Cost (in USD)', 'Adwords Keywords', 
                                     'Adwords Traffic', 'Adwords Buget (in USD)'])

In [24]:
print(extractedData.head())

                     Website Alexa Rank Back Links Page Authority  \
0     http://www.walmart.com        126          0          0/100   
1      http://www.amazon.com         14          0          0/100   
2  http://www.exxonmobil.com      27230          0          0/100   
3       http://www.apple.com         51          0          0/100   
4   http://www.cvshealth.com      31978          0          0/100   

  Domain Authority Moz Rank    Pageviews    Worth Page Size  \
0            0/100     0/10   8.73M/ Day   19.12M      64Kb   
1            0/100     0/10  78.57M/ Day  172.07M      64Kb   
2            0/100     0/10   40.4K/ Day   88.47K      64Kb   
3            0/100     0/10  21.57M/ Day   47.24M      64Kb   
4            0/100     0/10   34.4K/ Day   75.33K      64Kb   

                                  Code to Text Ratio Semrush Rank  Keywords  \
0    0.68% (Text size 0.44Kb and  Code size 63.56Kb)           15  29145287   
1    0.27% (Text size 0.17Kb and  Code size 63.8

In [25]:
extractedData.fillna(value=pd.np.nan, inplace=True)
extractedData = extractedData.replace({'\$': '', '/ Day': ''}, regex=True)

print(extractedData.head())

                     Website Alexa Rank Back Links Page Authority  \
0     http://www.walmart.com        126          0          0/100   
1      http://www.amazon.com         14          0          0/100   
2  http://www.exxonmobil.com      27230          0          0/100   
3       http://www.apple.com         51          0          0/100   
4   http://www.cvshealth.com      31978          0          0/100   

  Domain Authority Moz Rank Pageviews    Worth Page Size  \
0            0/100     0/10     8.73M   19.12M      64Kb   
1            0/100     0/10    78.57M  172.07M      64Kb   
2            0/100     0/10     40.4K   88.47K      64Kb   
3            0/100     0/10    21.57M   47.24M      64Kb   
4            0/100     0/10     34.4K   75.33K      64Kb   

                                  Code to Text Ratio Semrush Rank  Keywords  \
0    0.68% (Text size 0.44Kb and  Code size 63.56Kb)           15  29145287   
1    0.27% (Text size 0.17Kb and  Code size 63.83Kb)            3 

In [26]:
# define two functions to convert the suffixes 'K', 'M', 'B', 'Kb', 'Mb' in numbers
def textToNumber(val):
    intensity = {'K': 3, 'M': 6, 'B': 9}
    
    # check if the val is a number
    if isinstance(val, numbers.Number):
        return Decimal(val)
    
    if val[-1] in intensity:
        number, expo = val[:-1], val[-1]
        return Decimal(number) * 10 ** intensity[expo]
    else:
        return Decimal(val)
    
def computTextToNumber(val):
    intensity = {'Kb': 1, 'Mb': 2}

    # check if the val is a number
    if isinstance(val, numbers.Number):
        return Decimal(val)

    if val[-2:] in intensity:
        number, expo = val[:-2], val[-2:]
        return Decimal(number) * 2 ** (10 * intensity[expo])
    else:
        return Decimal(val)

In [27]:
# convert the 'numbers + suffix' in pure numbers
extractedData['Pageviews'] = extractedData['Pageviews'].apply(textToNumber)
extractedData['Worth'] = extractedData['Worth'].apply(textToNumber)
extractedData['Page Size'] = extractedData['Page Size'].apply(computTextToNumber)

In [28]:
# read the data from previous step
newFortune1000 = pd.read_csv('dfNewFortune1000.csv')

In [29]:
newFortune1000.head()

,Unnamed: 0,Rank,Name,Revenue($m),Profits($m),Assets($m),Market Value($m),Employees,Website
0,0,1,Walmart,523964.0,14881,236495.0,321803.3,2200000,http://www.walmart.com
1,1,2,Amazon.com,280522.0,11588,225248.0,970680.1,798000,http://www.amazon.com
2,2,3,Exxon Mobil,264938.0,14340,362597.0,160696.3,74900,http://www.exxonmobil.com
3,3,4,Apple,260174.0,55256,338516.0,1112640.8,137000,http://www.apple.com
4,4,5,CVS Health,256776.0,6634,222449.0,77375.8,290000,http://www.cvshealth.com


In [30]:
# merge two dataframes
newFortune1000 = newFortune1000.merge(extractedData, on='Website', how='left')

In [31]:
print(newFortune1000.head(10))

   Unnamed: 0  Rank                Name  Revenue($m) Profits($m)  Assets($m)  \
0           0     1             Walmart     523964.0       14881    236495.0   
1           1     2          Amazon.com     280522.0       11588    225248.0   
2           2     3         Exxon Mobil     264938.0       14340    362597.0   
3           3     4               Apple     260174.0       55256    338516.0   
4           4     5          CVS Health     256776.0        6634    222449.0   
5           5     6  Berkshire Hathaway     254616.0       81417    817729.0   
6           6     7  UnitedHealth Group     242155.0       13839    173889.0   
7           7     8            McKesson     214319.0          34     59672.0   
8           8     9                AT&T     181193.0       13903    551669.0   
9           9    10   AmerisourceBergen     179589.1       855.4     39172.0   

  Market Value($m)  Employees                           Website Alexa Rank  \
0         321803.3    2200000            

In [32]:
newFortune1000.drop(['Unnamed: 0'], inplace=True, axis=1)

In [33]:
print(newFortune1000.head(10))

   Rank                Name  Revenue($m) Profits($m)  Assets($m)  \
0     1             Walmart     523964.0       14881    236495.0   
1     2          Amazon.com     280522.0       11588    225248.0   
2     3         Exxon Mobil     264938.0       14340    362597.0   
3     4               Apple     260174.0       55256    338516.0   
4     5          CVS Health     256776.0        6634    222449.0   
5     6  Berkshire Hathaway     254616.0       81417    817729.0   
6     7  UnitedHealth Group     242155.0       13839    173889.0   
7     8            McKesson     214319.0          34     59672.0   
8     9                AT&T     181193.0       13903    551669.0   
9    10   AmerisourceBergen     179589.1       855.4     39172.0   

  Market Value($m)  Employees                           Website Alexa Rank  \
0         321803.3    2200000            http://www.walmart.com        126   
1         970680.1     798000             http://www.amazon.com         14   
2         160696.

In [34]:
newFortune1000.shape

(710, 24)

In [35]:
newFortune1000.to_csv('dfPart1_final.csv', index=False)